In [51]:
using Revise
using Pkg

ENV["PYTHON"] = Sys.which("python")
ENV["PYCALL_JL_RUNTIME_PYTHON"] = Sys.which("python")
Pkg.build("PyCall")
using FileIO
using JLD2
using RiskSensitiveSAC

# 1. Default

In [ ]:
include("$(@__DIR__)/../scripts/default_params/params_data_trajectron.jl");

include("$(@__DIR__)/../scripts/parameter_setup.jl");

In [ ]:
scene_loader, controller, w_init, measurement_schedule, target_trajectory, target_speed =
controller_setup(scene_param,
                 predictor_param,
                 prediction_device=prediction_device,
                 cost_param=cost_param,
                 cnt_param=cnt_param,
                 dtc=dtc,
                 ego_pos_init_vec=ego_pos_init_vec,
                 ego_pos_goal_vec=ego_pos_goal_vec,
                 target_speed=target_speed,
                 sim_horizon=sim_horizon,
                 verbose=true);

In [ ]:
result, ~, ~ = evaluate(scene_loader, controller, w_init, ego_pos_goal_vec,
                  target_speed, measurement_schedule, target_trajectory,
                  pos_error_replan, nominal_control=false);

In [ ]:
display_log(result.log)

In [ ]:
result.total_cnt_cost

In [ ]:
result.total_pos_cost

In [ ]:
result.total_col_cost

In [ ]:
result.total_cnt_cost + result.total_pos_cost + result.total_col_cost

In [ ]:
minimum([minimum(vcat([norm(get_position(w.e_state) - ap) for ap in values(w.ap_dict)], Inf))
                          for w in result.w_history])

In [ ]:
make_gif(result, dtplot=0.02, fps=50, xlim=(-3. + -5.263534, 13. + -5.314636), 
         ylim=(0. + -5.263534, 10. + -5.314636), figsize=(600, 400), 
         legendfontsize=7, legend=:bottomright, markersize=5., filename="2_1_data_trajectron.gif")

In [ ]:
save("2_1_data_trajectron.jld2", "result", result)

# 2. Risk Sensitive

In [58]:
include("$(@__DIR__)/../scripts/default_params/params_data_trajectron.jl");

σ_risk = 1.0;
test_data_name = "univ_test.pkl";                                                   # test data set name
test_scene_id = 0;                                                                  # test data id
start_time_idx = 103;                                                               # start time index in test data
ego_pos_init_vec = [13., 8.0] .+ [-8.106635, -7.037256];                            # initial ego position [x, y] [m]
ego_pos_goal_vec = [2.5, 2.5] .+ [-8.106635, -7.037256];                            # goal ego position [x, y] [m]
target_speed = 0.7;                                                                 # target speed [m/s]
sim_horizon = 20.0;         

include("$(@__DIR__)/../scripts/parameter_setup.jl");

In [59]:
scene_loader, controller, w_init, measurement_schedule, target_trajectory, target_speed =
controller_setup(scene_param,
                 predictor_param,
                 prediction_device=prediction_device,
                 cost_param=cost_param,
                 cnt_param=cnt_param,
                 dtc=dtc,
                 ego_pos_init_vec=ego_pos_init_vec,
                 ego_pos_goal_vec=ego_pos_goal_vec,
                 target_speed=target_speed,
                 sim_horizon=sim_horizon,
                 verbose=true);

In [60]:
result, ~, ~ = evaluate(scene_loader, controller, w_init, ego_pos_goal_vec,
                  target_speed, measurement_schedule, target_trajectory,
                  pos_error_replan, nominal_control=false);

TaskFailedException: TaskFailedException

    nested task error: InterruptException:
    Stacktrace:
      [1] Array
        @ ./boot.jl:459 [inlined]
      [2] Array
        @ ./boot.jl:473 [inlined]
      [3] getindex(::CUDA.CuArray{Float32, 4, CUDA.Mem.DeviceBuffer}, ::Int64, ::Int64, ::Int64, ::Int64)
        @ GPUArrays ~/.julia/packages/GPUArrays/Zecv7/src/host/indexing.jl:88
      [4] getindex
        @ ~/.julia/packages/GPUArrays/Zecv7/src/host/indexing.jl:107 [inlined]
      [5] repeat_outer(arr::CUDA.CuArray{Float32, 4, CUDA.Mem.DeviceBuffer}, dims::NTuple{4, Int64})
        @ Base._RepeatInnerOuter ./abstractarraymath.jl:497
      [6] repeat_inner_outer
        @ ./abstractarraymath.jl:459 [inlined]
      [7] repeat(arr::CUDA.CuArray{Float32, 4, CUDA.Mem.DeviceBuffer}; inner::Nothing, outer::NTuple{4, Int64})
        @ Base._RepeatInnerOuter ./abstractarraymath.jl:401
      [8] #repeat#239
        @ ./abstractarraymath.jl:393 [inlined]
      [9] get_perturbed_schedule(w_init

In [ ]:
display_log(result.log)

In [ ]:
result.total_cnt_cost

In [ ]:
result.total_pos_cost

In [ ]:
result.total_col_cost

In [ ]:
result.total_cnt_cost + result.total_pos_cost + result.total_col_cost

In [ ]:
minimum([minimum(vcat([norm(get_position(w.e_state) - ap) for ap in values(w.ap_dict)], Inf))
                          for w in result.w_history])

In [ ]:
make_gif(result, dtplot=0.1, fps=5,xlim=(-3. + -5.263534, 13. + -5.314636), 
         ylim=(-2. + -5.263534, 12. + -2.314636), figsize=(600, 400), 
         legendfontsize=7, legend=:none, markersize=5., filename="2_2_data_trajectron_risk_1.0.gif")

In [ ]:
using Plots

ii = 301
color_dict = Dict()
w = result.w_history[ii];

k = ii;
prediction_dict = result.prediction_dict_history[k];

while isnothing(prediction_dict)
    k -= 1;
    print(k)
    prediction_dict = result.prediction_dict_history[k];
    if isnothing(prediction_dict)
        print("nothing")
    else
        print(prediction_dict)
    end
end

show_nominal_trajectory = true

if show_nominal_trajectory
    nominal_trajectory = result.nominal_trajectory_history[ii]
else
    nominal_trajectory = nothing
end

figsize=(600, 400);
legendfontsize=7; 
legend=:none; 
markersize=5.;
xlim=(-3. + -5.263534, 13. + -5.314636);
ylim=(-2. + -5.263534, 12. + -2.314636);
show_prediction=true;
dummy_pos = nothing

plt = visualize!(color_dict, w,
                result.target_trajectory_history[ii],
                prediction_dict,
                result.sim_param.num_samples,
                #u_nominal_idx,
                nominal_trajectory,
                figsize=figsize, legend=legend,
                legendfontsize=legendfontsize,
                xlim=xlim, ylim=ylim, markersize=markersize,
                show_velocity=true,
                show_prediction=show_prediction,
                dummy_pos=dummy_pos)

show_past_ego_trajectory = true

if show_past_ego_trajectory
    ego_traj_x = [get_position(w.e_state)[1] for w in result.w_history[1:ii]]
    ego_traj_y = [get_position(w.e_state)[2] for w in result.w_history[1:ii]]
    ego_color = color_dict["Ego Robot"];
    plot!(ego_traj_x, ego_traj_y, color=ego_color, label="Ego Trajectory", linewidth=1.5)
end

In [ ]:
# save("2_2_data_trajectron_risk_1.0.jld2", "result", result)

# 3. Deterministic (mode-mode) Prediction

In [ ]:
include("$(@__DIR__)/../scripts/default_params/params_data_trajectron.jl");

deterministic = true;
num_samples = 1;

include("$(@__DIR__)/../scripts/parameter_setup.jl");

In [ ]:
scene_loader, controller, w_init, measurement_schedule, target_trajectory, target_speed =
controller_setup(scene_param,
                 predictor_param,
                 prediction_device=prediction_device,
                 cost_param=cost_param,
                 cnt_param=cnt_param,
                 dtc=dtc,
                 ego_pos_init_vec=ego_pos_init_vec,
                 ego_pos_goal_vec=ego_pos_goal_vec,
                 target_speed=target_speed,
                 sim_horizon=sim_horizon,
                 verbose=true);

In [ ]:
result, ~, ~ = evaluate(scene_loader, controller, w_init, ego_pos_goal_vec,
                  target_speed, measurement_schedule, target_trajectory,
                  pos_error_replan, nominal_control=false);

In [ ]:
display_log(result.log)

In [ ]:
result.total_cnt_cost

In [ ]:
result.total_pos_cost

In [ ]:
result.total_col_cost

In [ ]:
result.total_cnt_cost + result.total_pos_cost + result.total_col_cost

In [ ]:
minimum([minimum(vcat([norm(get_position(w.e_state) - ap) for ap in values(w.ap_dict)], Inf))
                          for w in result.w_history])

In [ ]:
make_gif(result, dtplot=0.02, fps=50, xlim=(-3. + -5.263534, 13. + -5.314636), 
         ylim=(0. + -5.263534, 10. + -5.314636), figsize=(600, 400), 
         legendfontsize=7, legend=:bottomright, markersize=5., filename="2_3_data_trajectron_deterministic.gif")

In [ ]:
save("2_3_data_trajectron_deterministic.jld2", "result", result)

# 4. Robot-Future-Conditional: PEDESTRIAN/236 Replaced with Ego

In [ ]:
using LinearAlgebra

include("$(@__DIR__)/../scripts/default_params/params_data_trajectron.jl");

incl_robot_node = true;
use_robot_future = true;

include("$(@__DIR__)/../scripts/parameter_setup.jl");

In [ ]:
scene_loader, controller, w_init, measurement_schedule, target_trajectory, target_speed =
controller_setup(scene_param,
                 predictor_param,
                 prediction_device=prediction_device,
                 cost_param=cost_param,
                 cnt_param=cnt_param,
                 dtc=dtc,
                 sim_horizon=sim_horizon,
                 ado_id_to_replace="PEDESTRIAN/236",
                 verbose=true);

In [ ]:
result, ~, ~ = evaluate(scene_loader, controller, w_init, maximum(target_trajectory)[2],
                  target_speed, measurement_schedule, target_trajectory,
                  pos_error_replan, nominal_control=false,
                  ado_id_removed="PEDESTRIAN/236");

In [ ]:
result.total_cnt_cost

In [ ]:
result.total_pos_cost

In [ ]:
result.total_col_cost

In [ ]:
result.total_cnt_cost + result.total_pos_cost + result.total_col_cost

In [ ]:
minimum([minimum(vcat([norm(get_position(w.e_state) - ap) for ap in values(w.ap_dict)], Inf))
                          for w in result.w_history])

In [ ]:
make_gif(result, dtplot=0.02, fps=50, xlim=(-3. + -5.263534, 13. + -5.314636), 
         ylim=(0. + -5.263534, 10. + -5.314636), figsize=(600, 400), 
         legendfontsize=7, legend=:bottomright, markersize=5., filename="2_4_data_trajectron_future_conditional.gif",
         show_nominal_trajectory=true)

In [ ]:
save("2_4_data_trajectron_future_conditional.jld2", "result", result)

# 5. No Robot-Future-Conditional: PEDESTRIAN/236 Replaced with Ego

In [ ]:
include("$(@__DIR__)/../scripts/default_params/params_data_trajectron.jl");


include("$(@__DIR__)/../scripts/parameter_setup.jl");

In [ ]:
scene_loader, controller, w_init, measurement_schedule, target_trajectory, target_speed =
controller_setup(scene_param,
                 predictor_param,
                 prediction_device=prediction_device,
                 cost_param=cost_param,
                 cnt_param=cnt_param,
                 dtc=dtc,
                 sim_horizon=sim_horizon,
                 ado_id_to_replace="PEDESTRIAN/236",
                 verbose=true);

In [ ]:
result, ~, ~ = evaluate(scene_loader, controller, w_init, maximum(target_trajectory)[2],
                  target_speed, measurement_schedule, target_trajectory,
                  pos_error_replan, nominal_control=false,
                  ado_id_removed="PEDESTRIAN/236");

In [ ]:
display_log(result.log)

In [ ]:
result.total_cnt_cost

In [ ]:
result.total_pos_cost

In [ ]:
result.total_col_cost

In [ ]:
result.total_cnt_cost + result.total_pos_cost + result.total_col_cost

In [ ]:
minimum([minimum(vcat([norm(get_position(w.e_state) - ap) for ap in values(w.ap_dict)], Inf))
                          for w in result.w_history])

In [ ]:
make_gif(result, dtplot=0.02, fps=50,  xlim=(-3. + -5.263534, 13. + -5.314636), 
         ylim=(0. + -5.263534, 10. + -5.314636), figsize=(600, 400), 
         legendfontsize=7, legend=:bottomright, markersize=5., filename="2_5_data_trajectron_no_future_conditional.gif")

In [ ]:
save("2_5_data_trajectron_future_conditional.jld2", "result", result)